In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from scipy import sparse

# 读取处理好的数据

In [2]:
from scipy import sparse
data_x=sparse.load_npz('./data/data_x_sparse.npz')

In [3]:
data_x=data_x[:,1:]  ##删id

In [4]:
data_x.shape

(100000, 6817)

In [2]:

data_tag=pd.read_csv('./shuchu/data_tag.csv')

In [9]:
sum(data_tag.tag==1)

30835

In [7]:
column=pd.read_csv('./shuchu/index.csv')
index=list(column["0"])
index.remove('id')
index.remove('label')

In [8]:
len(index)

6817

In [9]:
data_y=pd.read_csv("./data/data_y.csv",header=None)
data_y=data_y[1]

# xgboost函数

In [5]:

import xgboost as xgb



from sklearn.cross_validation import train_test_split
params={'booster':'gbtree',
    'objective': 'binary:logistic',
    'eval_metric':'auc',
    'gamma':0.1,
    'min_child_weight':1.1,
    'max_depth':7,
    'lambda':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'colsample_bylevel':0.7,
    'eta': 0.01,
    'tree_method':'exact',
    'seed':1000,
    'nthread':12
    }
plst = list(params.items())

def mxgboost(train_x,train_y,test_x,test_y,plst):
    xgb_train = xgb.DMatrix(train_x, label=train_y)
    xgb_test = xgb.DMatrix(test_x, label=test_y)
    del(train_x,train_y,test_x,test_y)
    watchlist = [(xgb_train, 'train'),(xgb_test, 'val')]
    model=xgb.train(plst, xgb_train,num_boost_round=1000,evals=watchlist,early_stopping_rounds=10)
    return model
def mpre_xgb(model,data):
    xgb_data=xgb.DMatrix(data)
    return model.predict(xgb_data)
#model.save_model('xgb.model')

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 分层bootstrap，及拟合xgboost基模型

In [ ]:

for i in range(300):
    row_1=data_tag[data_tag.tag==1].sample(n=25000,random_state=2018*i,axis=0,replace=True).index
    row_0=data_tag[data_tag.tag==0].sample(n=5000,random_state=2017*i,axis=0,replace=True).index
    
    data_x_train_tem=sparse.vstack([data_x[row_1,:],data_x[row_0,:]])
    data_y_train_tem=data_y[row_1].append(data_y[row_0])
    row_test=data_tag[data_tag.tag==1].index^row_1
    data_x_test_tem=data_x[row_test]
    data_y_test_tem=data_y[row_test]
    model_tem=mxgboost(data_x_train_tem,data_y_train_tem,data_x_test_tem,data_y_test_tem,plst)
    model_tem.save_model('xgb_'+str(i)+'.model')
    #pre.append(mpre_xgb(model_tem,test_online))
    print(i)

[0]	train-auc:0.675636	val-auc:0.639906
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 10 rounds.
[1]	train-auc:0.705267	val-auc:0.654296
[2]	train-auc:0.71585	val-auc:0.662936
[3]	train-auc:0.750251	val-auc:0.683456
[4]	train-auc:0.754157	val-auc:0.682564
[5]	train-auc:0.760071	val-auc:0.687614
[6]	train-auc:0.776723	val-auc:0.702438
[7]	train-auc:0.775983	val-auc:0.702038
[8]	train-auc:0.779015	val-auc:0.702071
[9]	train-auc:0.787892	val-auc:0.705311
[10]	train-auc:0.788389	val-auc:0.706923
[11]	train-auc:0.790093	val-auc:0.70768
[12]	train-auc:0.797561	val-auc:0.712005
[13]	train-auc:0.799053	val-auc:0.712215
[14]	train-auc:0.802581	val-auc:0.718058
[15]	train-auc:0.806122	val-auc:0.717814
[16]	train-auc:0.812094	val-auc:0.725905
[17]	train-auc:0.816113	val-auc:0.725175
[18]	train-auc:0.821082	val-auc:0.726741
[19]	train-auc:0.823932	val-auc:0.726777
[20]	train-auc:0.824407	val-auc:0.726843
[21]	train-a

[196]	train-auc:0.934004	val-auc:0.776517
[197]	train-auc:0.93433	val-auc:0.776818
[198]	train-auc:0.934878	val-auc:0.777204
[199]	train-auc:0.935438	val-auc:0.777666
[200]	train-auc:0.935701	val-auc:0.777836
[201]	train-auc:0.936005	val-auc:0.778326
[202]	train-auc:0.936062	val-auc:0.77837
[203]	train-auc:0.936394	val-auc:0.778458
[204]	train-auc:0.93677	val-auc:0.77852
[205]	train-auc:0.937191	val-auc:0.778864
[206]	train-auc:0.937353	val-auc:0.77937
[207]	train-auc:0.937604	val-auc:0.779758
[208]	train-auc:0.937862	val-auc:0.780203
[209]	train-auc:0.938194	val-auc:0.780371
[210]	train-auc:0.938471	val-auc:0.780867
[211]	train-auc:0.93883	val-auc:0.781669
[212]	train-auc:0.939265	val-auc:0.782125
[213]	train-auc:0.939621	val-auc:0.782385
[214]	train-auc:0.940024	val-auc:0.782847
[215]	train-auc:0.94028	val-auc:0.783031
[216]	train-auc:0.940697	val-auc:0.783331
[217]	train-auc:0.940876	val-auc:0.783398
[218]	train-auc:0.941245	val-auc:0.783688
[219]	train-auc:0.941776	val-auc:0.784005

[393]	train-auc:0.979334	val-auc:0.807642
[394]	train-auc:0.979453	val-auc:0.807921
[395]	train-auc:0.979543	val-auc:0.807929
[396]	train-auc:0.979649	val-auc:0.80793
[397]	train-auc:0.979691	val-auc:0.807935
[398]	train-auc:0.979773	val-auc:0.808084
[399]	train-auc:0.979873	val-auc:0.808105
[400]	train-auc:0.979962	val-auc:0.808135
[401]	train-auc:0.980104	val-auc:0.808247
[402]	train-auc:0.980204	val-auc:0.808386
[403]	train-auc:0.980345	val-auc:0.808396
[404]	train-auc:0.980511	val-auc:0.808432
[405]	train-auc:0.980658	val-auc:0.808592
[406]	train-auc:0.980812	val-auc:0.80873
[407]	train-auc:0.980963	val-auc:0.808769
[408]	train-auc:0.981133	val-auc:0.80895
[409]	train-auc:0.98124	val-auc:0.809019
[410]	train-auc:0.98133	val-auc:0.809088
[411]	train-auc:0.981471	val-auc:0.809092
[412]	train-auc:0.981559	val-auc:0.809229
[413]	train-auc:0.981642	val-auc:0.809368
[414]	train-auc:0.981764	val-auc:0.809361
[415]	train-auc:0.981905	val-auc:0.809448
[416]	train-auc:0.981993	val-auc:0.8096

[589]	train-auc:0.993886	val-auc:0.820198
[590]	train-auc:0.993944	val-auc:0.820174
[591]	train-auc:0.993977	val-auc:0.820248
[592]	train-auc:0.994024	val-auc:0.82042
[593]	train-auc:0.99406	val-auc:0.820466
[594]	train-auc:0.994091	val-auc:0.820539
[595]	train-auc:0.994118	val-auc:0.820642
[596]	train-auc:0.994152	val-auc:0.820716
[597]	train-auc:0.99418	val-auc:0.820741
[598]	train-auc:0.9942	val-auc:0.820737
[599]	train-auc:0.994232	val-auc:0.820713
[600]	train-auc:0.994254	val-auc:0.820724
[601]	train-auc:0.994283	val-auc:0.820813
[602]	train-auc:0.994312	val-auc:0.820861
[603]	train-auc:0.994377	val-auc:0.820908
[604]	train-auc:0.994406	val-auc:0.820908
[605]	train-auc:0.994436	val-auc:0.820912
[606]	train-auc:0.99449	val-auc:0.821004
[607]	train-auc:0.994525	val-auc:0.821019
[608]	train-auc:0.99455	val-auc:0.820991
[609]	train-auc:0.994584	val-auc:0.820964
[610]	train-auc:0.994623	val-auc:0.820932
[611]	train-auc:0.994656	val-auc:0.820997
[612]	train-auc:0.994692	val-auc:0.820934

[103]	train-auc:0.888231	val-auc:0.775606
[104]	train-auc:0.889846	val-auc:0.775792
[105]	train-auc:0.890191	val-auc:0.776312
[106]	train-auc:0.891031	val-auc:0.777144
[107]	train-auc:0.892576	val-auc:0.777251
[108]	train-auc:0.893528	val-auc:0.777663
[109]	train-auc:0.894645	val-auc:0.778338
[110]	train-auc:0.895454	val-auc:0.77872
[111]	train-auc:0.896332	val-auc:0.779144
[112]	train-auc:0.897151	val-auc:0.779781
[113]	train-auc:0.897627	val-auc:0.779668
[114]	train-auc:0.898402	val-auc:0.780082
[115]	train-auc:0.89935	val-auc:0.780433
[116]	train-auc:0.900924	val-auc:0.781094
[117]	train-auc:0.90174	val-auc:0.781388
[118]	train-auc:0.902263	val-auc:0.78162
[119]	train-auc:0.903041	val-auc:0.782097
[120]	train-auc:0.903589	val-auc:0.782431
[121]	train-auc:0.904267	val-auc:0.782447
[122]	train-auc:0.904942	val-auc:0.782694
[123]	train-auc:0.905222	val-auc:0.782922
[124]	train-auc:0.905417	val-auc:0.782916
[125]	train-auc:0.905672	val-auc:0.7828
[126]	train-auc:0.906183	val-auc:0.78292

...

[167]	train-auc:0.918411	val-auc:0.777802
[168]	train-auc:0.918516	val-auc:0.77761
[169]	train-auc:0.919081	val-auc:0.777391
[170]	train-auc:0.919437	val-auc:0.777852
[171]	train-auc:0.919976	val-auc:0.777954
[172]	train-auc:0.920199	val-auc:0.778442
[173]	train-auc:0.92108	val-auc:0.778633
[174]	train-auc:0.921447	val-auc:0.778845
[175]	train-auc:0.922037	val-auc:0.77884
[176]	train-auc:0.922441	val-auc:0.779161
[177]	train-auc:0.922548	val-auc:0.779203
[178]	train-auc:0.922813	val-auc:0.779491
[179]	train-auc:0.9232	val-auc:0.779629
[180]	train-auc:0.923865	val-auc:0.779895
[181]	train-auc:0.924148	val-auc:0.780177
[182]	train-auc:0.924629	val-auc:0.780511
[183]	train-auc:0.925225	val-auc:0.780684
[184]	train-auc:0.925615	val-auc:0.78112
[185]	train-auc:0.925987	val-auc:0.781114
[186]	train-auc:0.926291	val-auc:0.781269
[187]	train-auc:0.927098	val-auc:0.781794
[188]	train-auc:0.927218	val-auc:0.781783
[189]	train-auc:0.927961	val-auc:0.781516
[190]	train-auc:0.928118	val-auc:0.78175

[364]	train-auc:0.975028	val-auc:0.811481
[365]	train-auc:0.975251	val-auc:0.811606
[366]	train-auc:0.975459	val-auc:0.811607
[367]	train-auc:0.975647	val-auc:0.811727
[368]	train-auc:0.975661	val-auc:0.811863
[369]	train-auc:0.975815	val-auc:0.81204
[370]	train-auc:0.975982	val-auc:0.811998
[371]	train-auc:0.976077	val-auc:0.81217
[372]	train-auc:0.976244	val-auc:0.812376
[373]	train-auc:0.976388	val-auc:0.812408
[374]	train-auc:0.976557	val-auc:0.81258
[375]	train-auc:0.976742	val-auc:0.812583
[376]	train-auc:0.976873	val-auc:0.812623
[377]	train-auc:0.97703	val-auc:0.812672
[378]	train-auc:0.977151	val-auc:0.81281
[379]	train-auc:0.977203	val-auc:0.812936
[380]	train-auc:0.977363	val-auc:0.812965
[381]	train-auc:0.977518	val-auc:0.813185
[382]	train-auc:0.977601	val-auc:0.813181
[383]	train-auc:0.977728	val-auc:0.813391
[384]	train-auc:0.97789	val-auc:0.813368
[385]	train-auc:0.97793	val-auc:0.813495
[386]	train-auc:0.978069	val-auc:0.813488
[387]	train-auc:0.978201	val-auc:0.8135
[

[48]	train-auc:0.828448	val-auc:0.740727
[49]	train-auc:0.835093	val-auc:0.75037
[50]	train-auc:0.835072	val-auc:0.751132
[51]	train-auc:0.836344	val-auc:0.750606
[52]	train-auc:0.839622	val-auc:0.754837
[53]	train-auc:0.840558	val-auc:0.756043
[54]	train-auc:0.84323	val-auc:0.75721
[55]	train-auc:0.844903	val-auc:0.758709
[56]	train-auc:0.845936	val-auc:0.759064
[57]	train-auc:0.846485	val-auc:0.759563
[58]	train-auc:0.847454	val-auc:0.759352
[59]	train-auc:0.847822	val-auc:0.758803
[60]	train-auc:0.850779	val-auc:0.759696
[61]	train-auc:0.85149	val-auc:0.759338
[62]	train-auc:0.851758	val-auc:0.75979
[63]	train-auc:0.85186	val-auc:0.759928
[64]	train-auc:0.853883	val-auc:0.761158
[65]	train-auc:0.85464	val-auc:0.761192
[66]	train-auc:0.8547	val-auc:0.761669
[67]	train-auc:0.855799	val-auc:0.761754
[68]	train-auc:0.85635	val-auc:0.762472
[69]	train-auc:0.856372	val-auc:0.762595
[70]	train-auc:0.856923	val-auc:0.762544
[71]	train-auc:0.856962	val-auc:0.762747
[72]	train-auc:0.858252	va

[246]	train-auc:0.953768	val-auc:0.80671
[247]	train-auc:0.95433	val-auc:0.806729
[248]	train-auc:0.954569	val-auc:0.807038
[249]	train-auc:0.954895	val-auc:0.807185
[250]	train-auc:0.955276	val-auc:0.807435
[251]	train-auc:0.95566	val-auc:0.807598
[252]	train-auc:0.95578	val-auc:0.807809
[253]	train-auc:0.95597	val-auc:0.807925
[254]	train-auc:0.956213	val-auc:0.808189
[255]	train-auc:0.956483	val-auc:0.808372
[256]	train-auc:0.95677	val-auc:0.80857
[257]	train-auc:0.957329	val-auc:0.808854
[258]	train-auc:0.957559	val-auc:0.808852
[259]	train-auc:0.957808	val-auc:0.809002
[260]	train-auc:0.958059	val-auc:0.80922
[261]	train-auc:0.958307	val-auc:0.809425
[262]	train-auc:0.958654	val-auc:0.809698
[263]	train-auc:0.958854	val-auc:0.809866
[264]	train-auc:0.95909	val-auc:0.80988
[265]	train-auc:0.959326	val-auc:0.809982
[266]	train-auc:0.959496	val-auc:0.810063
[267]	train-auc:0.95989	val-auc:0.810002
[268]	train-auc:0.960236	val-auc:0.810326
[269]	train-auc:0.960468	val-auc:0.810318
[27

[443]	train-auc:0.986924	val-auc:0.825162
[444]	train-auc:0.986993	val-auc:0.825267
[445]	train-auc:0.987058	val-auc:0.825317
[446]	train-auc:0.987111	val-auc:0.825363
[447]	train-auc:0.987173	val-auc:0.825424
[448]	train-auc:0.987263	val-auc:0.825449
[449]	train-auc:0.987362	val-auc:0.825568
[450]	train-auc:0.987443	val-auc:0.825742
[451]	train-auc:0.987562	val-auc:0.825855
[452]	train-auc:0.987639	val-auc:0.825756
[453]	train-auc:0.987694	val-auc:0.825845
[454]	train-auc:0.987748	val-auc:0.825979
[455]	train-auc:0.98783	val-auc:0.825925
[456]	train-auc:0.987869	val-auc:0.82598
[457]	train-auc:0.987947	val-auc:0.826046
[458]	train-auc:0.988002	val-auc:0.826053
[459]	train-auc:0.988043	val-auc:0.826056
[460]	train-auc:0.988068	val-auc:0.826048
[461]	train-auc:0.988165	val-auc:0.826091
[462]	train-auc:0.988223	val-auc:0.826118
[463]	train-auc:0.988337	val-auc:0.826135
[464]	train-auc:0.988411	val-auc:0.826248
[465]	train-auc:0.988494	val-auc:0.826283
[466]	train-auc:0.988559	val-auc:0.8

# 测试集预处理

In [3]:

test_online=pd.read_table("test.txt")

import time, datetime
def ymd(d):
    a=np.zeros(d.shape[0])
    b=np.zeros(d.shape[0])
    c=np.zeros(d.shape[0])
    e=np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        timeArray = time.strptime(d.loan_dt[i], "%Y-%m-%d")
        a[i]=timeArray.tm_year
        b[i]=timeArray.tm_mon
        c[i]=timeArray.tm_mday
        e[i]=timeArray.tm_wday
    d['year']=a
    d['mon']=b
    d['mday']=c
    d['wday']=e
ymd(test_online)

#test_online.loan_dt
import time, datetime
def t(d):
    timeStamp=np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        timeArray = time.strptime(d[i], "%Y-%m-%d")
        #print(timeArray)     
        # 转为时间戳
        timeStamp[i] = (int(time.mktime(timeArray))//86400)
        #print (timeStamp)  # 1381419600
    #print(timeStamp)
    return timeStamp-17256.0

test_online.loan_dt=t(test_online.loan_dt)
test_online['tag']=1

#test_online['tag']=1
#xgb_online_test=xgb.DMatrix(test_online

def queshi(d):
    d['queshi']=np.isnan(d[:]).sum(axis=1)
queshi(test_online)

test_online=test_online.drop(['id'],axis=1)
column=pd.read_csv('./shuchu/index.csv')

index=list(column["0"])
index.remove('id')
index.remove('label')
test_online=sparse.csc_matrix(test_online[index])

# 挑选模型大于1M的对测试集预测

In [6]:
from os.path import getsize

pre=[]
for i in range(300):
    print("第"+str(i)+"轮循环")
    if (getsize('xgb_'+str(i)+'.model')/1024/1024)>1:
        model=xgb.Booster(model_file='xgb_'+str(i)+'.model')
        pre.append(mpre_xgb(model,test_online))
        print("选入")

第0轮循环
选入
第1轮循环
选入
第2轮循环
第3轮循环
选入
第4轮循环
选入
第5轮循环
第6轮循环
第7轮循环
第8轮循环
第9轮循环
第10轮循环
第11轮循环
第12轮循环
第13轮循环
第14轮循环
第15轮循环
选入
第16轮循环
第17轮循环
第18轮循环
第19轮循环
第20轮循环
第21轮循环
选入
第22轮循环
选入
第23轮循环
第24轮循环
选入
第25轮循环
第26轮循环
第27轮循环
选入
第28轮循环
选入
第29轮循环
第30轮循环
第31轮循环
第32轮循环
第33轮循环
第34轮循环
第35轮循环
第36轮循环
第37轮循环
第38轮循环
第39轮循环
选入
第40轮循环
选入
第41轮循环
第42轮循环
第43轮循环
第44轮循环
第45轮循环
选入
第46轮循环
选入
第47轮循环
选入
第48轮循环
选入
第49轮循环
第50轮循环
选入
第51轮循环
选入
第52轮循环
选入
第53轮循环
选入
第54轮循环
第55轮循环
选入
第56轮循环
选入
第57轮循环
选入
第58轮循环
选入
第59轮循环
第60轮循环
第61轮循环
第62轮循环
第63轮循环
第64轮循环
选入
第65轮循环
第66轮循环
选入
第67轮循环
第68轮循环
选入
第69轮循环
第70轮循环
第71轮循环
第72轮循环
第73轮循环
选入
第74轮循环
第75轮循环
选入
第76轮循环
选入
第77轮循环
第78轮循环
第79轮循环
第80轮循环
第81轮循环
选入
第82轮循环
第83轮循环
选入
第84轮循环
第85轮循环
选入
第86轮循环
第87轮循环
第88轮循环
第89轮循环
选入
第90轮循环
第91轮循环
第92轮循环
第93轮循环
第94轮循环
第95轮循环
选入
第96轮循环
选入
第97轮循环
选入
第98轮循环
第99轮循环
第100轮循环
选入
第101轮循环
选入
第102轮循环
选入
第103轮循环
选入
第104轮循环
第105轮循环
第106轮循环
第107轮循环
选入
第108轮循环
选入
第109轮循环
第110轮循环
选入
第111轮循环
选入
第112轮循环
第113轮循环
第114轮循环
选入
第115轮循环
第116轮循环
选入
第117轮循环
选入
第118轮循环
选入
第119轮循环
第12

In [7]:
len(pre)

167

In [9]:
max(np.mean(pre,axis=0))

0.6703671

In [11]:
"""
输出结果
"""
v_id=pd.read_table("test_id.txt")
f=pd.DataFrame(columns=["id","prob"])
f.prob=np.mean(pre,axis=0)
f.id=v_id
#print(f)
f.to_csv("test_xgb big than 1M.csv",encoding="utf-8")